## Here are some useful links for making your forecast!

**0. Climatology: What is the “average” like?**
* Temp/Precip: https://tinyurl.com/t69sym56

**1. Observations: What has happened recently?**
* Satellite Observations: https://www.star.nesdis.noaa.gov/GOES/conus_band.php?sat=G16&band=GEOCOLOR&length=24
* US Surface Analysis: https://tinyurl.com/jrhjfxp9
* Station Observations: https://tinyurl.com/39m9a4bz

**2. Model Data: What do the models predict?**
* Model Forecasts: https://www.tropicaltidbits.com/
* MOS Forecasts: https://vlab.noaa.gov/web/mdl/mos-products
* Model Forecast Summary: https://tinyurl.com/mrxmtjmf
* USL Model: http://www.microclimates.org/forecast/KTLH/

**3. Meteorologists: What do the human experts think?**
* Forecast Discussion: https://www.weather.gov/bgm/
* NWS Forecast: https://tinyurl.com/4cjv7wa5

**4. Your Forecast**
* https://www.wxchallenge.com/submit_forecast.php

In [ ]:
import os

station_state = 'MT'                # state identifier of the station (i.e. "WA" for Washington)
station_id = 'KMSO'                 # 4 character station ID (i.e. "KUIL" for Quillayute)

res = os.system('cd src; bash run.sh %s %s' % (station_state, station_id))

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
Image(url= "src//fig/Max_Temperature.png")

In [ ]:
Image(url= "src/fig/Min_Temperature.png")

In [ ]:
Image(url="src/fig/Precipitation.png")

In [ ]:
Image(url= "src/fig/2-Minute_Maximum_Wind_(knots).png")

In [ ]:
Image(url= "src/fig/Wind_Rose.png")

In [ ]:
Image(url= "src/fig/recent_TMax.png")

In [ ]:
Image(url= "src/fig/recent_TMin.png")

In [ ]:
Image(url= "src/fig/recent_wnd.png")

The below section will be useful when making your weather discussions. It will download GFS forecast data fields for you depending on what you set for the forecast initialization time, and the forecast hours. This is high resolution data, so be careful not to download too much.

In [ ]:
import datetime
import requests
import xarray as xr

# Fill out the forecast initialization time in the "dt" variable below.
dt = datetime.datetime(2024, 9, 24, 12)
# Fill out the hours since initialization of the forecast to download.
forecast_hours = [0, 12, 24, 36, 48]

# The ensuing code will download the GFS forecast fields based on your above selection.
date_keys = (dt.year, dt.month, dt.day, dt.hour)
url_link = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.%d%02d%02d/%02d/atmos' % (date_keys)

filenames = ['%s/gfs.t%02dz.pgrb2.0p25.f%03d' % (url_link, dt.hour, x) for x in forecast_hours]
for file in filenames:
    filename = file.strip()
    if not os.path.exists(os.path.basename(filename)):
        print('Downloading', file)
        req = requests.get(filename, allow_redirects=True)
        open(os.path.basename(filename), 'wb').write(req.content)
    else:
        print('Already found %s' % os.path.basename(filename))


In [ ]:
# The below code will read in the u, v, w, t, and geopotential height fields
# from the GFS forecast fields.
filelist_arr = [os.path.basename(file) for file in filenames]
ds = xr.open_mfdataset(filelist_arr, engine = 'cfgrib',
                       combine='nested', concat_dim = 'time',
                       filter_by_keys={'typeOfLevel': 'isobaricInhPa',
                                       'shortName': ['u', 'v', 'w', 't', 'gh']})